In [ ]:
import pandas as pd
from pathlib import Path
from sad2_final_project.analysis import add_missing_metrics_from_experiment, loader_obsolete_data, compute_wilcoxon_table, plot_wilcoxon_heatmap
import os
from sad2_final_project.analysis import plot_scatter, plot_boxplot, plot_scatter_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf


cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)
print(os.getcwd())
## create paths
DATA_PATH2 = Path('data/trajectory_length_vs_attractors1')
DATA_PATHS = Path('data/analysis_3_synchronous_part_1')
DATA_PATHA = Path('data/analysis_3_asynchronous_part_1')

df = loader_obsolete_data(DATA_PATH2 / 'results/metadata.csv', DATA_PATH2 / 'results/joined_results_trajectory_length_vs_attractors.csv')
# dfs = pd.read_csv(DATA_PATHS / 'results/metadata.csv')
# dfa = pd.read_csv(DATA_PATHA / 'results/metadata.csv')
dfs = loader_obsolete_data(DATA_PATHS / 'results/metadata.csv', DATA_PATHS / 'results/joined_results_analysis_3_synchronous.csv')
dfa = loader_obsolete_data(DATA_PATHA / 'results/metadata.csv', DATA_PATHA / 'results/joined_results_analysis_3_asynchronous.csv')

# print(dfs.columns)
# print(dfa.columns)
metrics_list=['TP', 'FP', 'FN', 'precision', 'recall', 'sensitivity', 'AHD', 'SHD', 'EHD', 'SID']
df = add_missing_metrics_from_experiment(df, DATA_PATH2, metrics_list, after_column='attractor_ratio')
dfs = add_missing_metrics_from_experiment(dfs, DATA_PATHS, metrics_list, after_column='attractor_ratio')
dfa = add_missing_metrics_from_experiment(dfa, DATA_PATHA, metrics_list, after_column='attractor_ratio')

# Introduction 
## Goal
The goal of this study is to investigate the relation between **sampling strategy** and **model selection metrics** in *boolean Bayesian networks (BDNs)* using `BNFinder2`. 

In particular, we analyze how characteristics of time-series data generated from Boolean networks influence the accuracy of reconstructing the underlying network structure using Bayesian methods. The experimental factors under investigation include:
- the **trajectory length**,
- the **sampling frequency**, defined as selecting every $n$-th state along a trajectory,
- the ratio between the **number of nodes** and the **trajectory length**, introduced as a normalization parameter $k$.

The generated datasets are grouped into classes determined by:
- the **update mode** (synchronous vs asynchronous),
- the **scoring function** used during inference (MDL and BDe).

In addition, we study **scaling relations with respect to the number of nodes**, aiming to characterize how data requirements and reconstruction accuracy change as network size increases.

# Part 1

## Methodology
### General reasoning 
The primary objective of the experimental design is to isolate how properties of sampled time-series data affect the accuracy of Boolean network reconstruction using `BNFinder2`. 
***
### Update Mode
We distinguish between two fundamentally different update mechanisms:
- **Synchronous update**, which defines a deterministic dynamical system: from any given state, the successor state is uniquely determined.
- **Asynchronous update**, which induces a stochastic process: at each time step, a randomly selected node is updated, leading to multiple possible successor states.
This distinction is critical, as asynchronous dynamics introduce temporal dependence and potential autocorrelation in trajectories. In particular, long residence times in attractors or local cycles may reduce the effective information content of sampled data. Consequently, naive dense sampling may lead to strongly correlated observations, while aggressive subsampling may destroy causal ordering information.
***
### Scoring Functions
We employ two scoring functions implemented in BNFinder2, which differ in how they trade off data fit against model complexity.
Let $G$ denote a candidate network structure and $D$ the observed dataset.

**Minimal Description Length (MDL)**

The MDL score is defined as
$$
\mathrm{MDL}(G \mid D)
= - \log P(D \mid G, \hat{\theta})
\times \frac{1}{2} , |\theta_G| , \log |D|,
$$


where $\hat{\theta}$ are maximum-likelihood parameters and $|\theta_G|$ denotes the number of free parameters implied by the graph structure $G$.

The first term rewards goodness of fit, while the second term penalizes model complexity. As a consequence, MDL favors simpler graphs when data are scarce and becomes less restrictive as sample size increases. This makes MDL particularly sensitive to undersampling and normalization effects.

**Bayesian–Dirichlet equivalence (BDe)**

The BDe score evaluates the marginal likelihood

$$
\mathrm{BDe}(G \mid D)
= \log \int P(D \mid G, \theta), P(\theta \mid G), d\theta,
$$

assuming a Dirichlet prior over conditional probability tables. Under standard assumptions, this integral has a closed-form.

Unlike MDL, BDe incorporates prior beliefs and smooths parameter estimates, which can stabilize inference in low-data regimes but may also reduce sensitivity to subtle structural differences.

By comparing MDL and BDe, we assess whether observed reconstruction effects are driven primarily by data properties or by the inductive bias of the scoring function.

One caveat is that our implementation of those functions is simplified compared to implemented in BNfinder, which may lead to differences in absolute performance. However, relative trends should remain consistent.

---

### Evaluation Metrics
Reconstruction quality is evaluated using structure-based graph distance measures. Each metric captures a distinct notion of discrepancy between the true network $G^\ast$ and the inferred network $\hat{G}$. 

**Adjusted Hamming Distance (AHD)**

Let $A^\ast$ and $\hat{A}$ denote the adjacency matrices of $G^\ast$ and $\hat{G}$. The adjusted Hamming distance is defined as

$$
\mathrm{AHD}(G^\ast, \hat{G})
= \frac{1}{|E^\ast| + |\hat{E}|}
\sum_{i,j} \mathbf{1}_{{A^\ast_{ij} \neq \hat{A}_{ij}}}.
$$
AHD measures the proportion of mismatched edges, normalized by graph size. It penalizes false positives and false negatives symmetrically and allows comparisons across networks of different sizes. This metric serves as the primary measure of structural accuracy.

**Structural Hamming Distance (SHD)**

SHD counts the minimum number of edge insertions, deletions, or reversals required to transform $\hat{G}$ into $G^\ast$.
$
\mathrm{SHD}(G^\ast, \hat{G}) \in \mathbb{N}.
$
While widely used, SHD aggregates heterogeneous error types and does not distinguish between missing, extra, or misoriented edges, limiting its interpretability.

**Structural Intervention Distance (SID)**

SID measures the number of node pairs $(i,j)$ for which the causal effect of intervening on $i$ differs between $G^\ast$ and $\hat{G}$.

$$
\mathrm{SID}(G^\ast, \hat{G})
= \left| {(i,j) : P(j \mid \mathrm{do}(i))*{G^\ast}
\neq P(j \mid \mathrm{do}(i))*{\hat{G}} } \right|.
$$
Where $\mathrm{do}(i=n)$ set node $i$ to have value $n$ at time step, regardles of Boolean update rule.

SID is sensitive to edge orientation and captures errors that affect causal interpretability, even when the undirected structure is largely correct.

Using these metrics jointly allows us to separate purely topological accuracy (AHD, SHD) from correctness of implied causal relationships (SID) and to identify metric-specific effects of sampling and model selection.

---
### Independence
To avoid introducing structural bias, Boolean networks are generated randomly for each condition:
- each node is assigned a random number of parents (uniformly chosen from $\{ 1 ,2 ,3 \}$),
- Boolean update functions are sampled randomly.
All generated networks are accepted without filtering. Repetitions under identical conditions are treated as independent realizations.
### Sample Size Normalization
To ensure comparability across networks of different sizes, we introduce **sample-size normalization**.
Let:
- nnodesn_{\text{nodes}}nnodes​ denote the number of nodes,
- TlengthT_{\text{length}}Tlength​ the trajectory length,
- kkk a normalization constant.
The total number of sampled time points is defined as:
$$
\begin{align}
T_{\text{amount}} & = \frac{{n_{\text{nodes}} \cdot k}}{T_{\text{length}}} 
\end{align}
$$

We fix $k=100$. This choice is motivated by the fact that each node with at most three parents has up to $2^3 = 8$ possible parent-state configurations. Setting $k=100$ corresponds to approximately 10 observations per configuration per node, providing a conservative buffer against stochastic effects and subsampling losses.

### Dataset Construction
#### General Settings
Across all experiments, we vary the following parameters:
* number of nodes (`num_nodes`): ([5, 7, 11, 13, 15]),
* scoring function (`score_function`): MDL, BDe,
* update mode (`update_mode`): synchronous, asynchronous,
* parent count per node: randomly chosen from ({1,2,3}),
* number of repetitions per condition: 30.
---
#### Dataset 1 (Baseline Dataset)
This dataset is used in Experiments 1 and 2.
* trajectory length (`trajectory_length`): ([10, 15, 20, 30, 40, 50]),
* sampling frequency (`sampling_frequency`): ([1, 2, 3, 4, 5]),
* number of trajectories: determined via sample-size normalization.
---
#### Dataset 1.1 (Low-Data Regime)
To probe behavior in extreme data-scarce settings, an auxiliary dataset is constructed with:
* trajectory length: ([5, 7, 9]),
* number of nodes: ([5, 7, 9]),
* sampling frequency: ([1, 2, 3, 4, 5]).
This dataset is included to assess scaling behavior when normalization constraints are tight.
---
#### Dataset 2 (Normalization Study)
Using optimal parameters identified in Experiments 1 and 2, we fix:

$$\begin{align*} \text{sampling frequency} & =\begin{cases}1  & \text{synchronous} \\ 4 & \text{asynchronous}\end{cases} & \mathrm{trajectory\ length}=[0.8 \cdot x]_{\mathrm{round}}\end{align*}$$

We then vary the normalization constant:
$$k \in \{20, 40, 60, 80, 100, 120, 140\}$$

---
### Experiments
#### Experiment 1: Attractor Prevalence and Trajectory Length
Motivated by prior observations that attractor-heavy datasets degrade reconstruction quality, we investigate:
1. the correlation between attractor fraction and reconstruction metrics,
2. how attractor prevalence depends on trajectory length,
3. how these effects scale with network size.
We introduce the **scale ratio**
$$\text{scale ratio} = \frac{T_{\text{length}}}{n_{\text{nodes}}}$$
and analyze its relationship with attractor fraction to identify regimes that balance coverage of transient dynamics and attractor exploration.

---
#### Experiment 2: Subsampling and Temporal Dependence
This experiment examines whether there exists relation between ESS and metrics score functions subsampling improves reconstruction quality by reducing temporal dependence.
We first analyze:
* **effective sample size (ESS)**,
* **autocorrelation factor (ACF)**,
as functions of update mode and sampling frequency.
We then assess monotonic relationships between ESS and reconstruction metrics using Spearman correlation. Statistical significance of performance differences between sampling frequencies is evaluated using paired Wilcoxon tests, applied only when sufficient paired observations are available.
To avoid confounding effects scale ratios below 1.5 are excluded and cases without effective subsampling are removed.
---
#### Experiment 3: Normalization and Information Saturation
Fixing sampling parameters based on earlier results, we analyze how reconstruction metrics evolve as a function of the normalization constant (k). This allows us to identify saturation regimes in which increasing sample size yields diminishing returns.

---
#### Experiment 4: Score Function Sensitivity
Finally, using a subset of Experiment 3 with fixed normalization, we investigate how reconstruction metrics depend on the choice of scoring function. The goal is to understand whether differences in reconstruction quality arise from data properties or from intrinsic characteristics of the scoring criteria.

## Analysis

### Experiment 1

##### Determinants of Reconstruction Accuracy

The primary objective of this analysis was to identify the key factors influencing the accuracy of Boolean network reconstruction using dynamic Bayesian networks (DBNs). Specifically, we investigated how properties of the generated data (attractor ratio, trajectory length), network characteristics (number of nodes), and inference configuration (update mode and scoring function) affect reconstruction quality as measured by standard performance metrics.

###### Exploratory Correlation Analysis

We first conducted a pairwise correlation analysis between data characteristics and reconstruction metrics (precision, recall, SHD, and AHD).

In [ ]:
df1 = df[df["trajectory_length"]>=10]
corr_matrix = df[['attractor_ratio', 'num_nodes', 'trajectory_length', 'precision', 'recall', 'SHD', 'AHD']].corr(method='spearman')
g = sns.clustermap(corr_matrix,
                   annot=True,
                   cmap="coolwarm",
                   fmt=".2f",
                   figsize=(10, 10),
                   row_cluster=True,
                   col_cluster=True,
                   center=0,
                   dendrogram_ratio=0.1,
                   cbar_pos=(0.02, 0.8, 0.03, 0.18))
g.ax_heatmap.set_title("Clustered Correlation Matrix", fontsize=16, pad=20)
plt.show()


This exploratory step revealed several notable relationships:

- **Attractor ratio** exhibited a strong negative correlation with recall (r = –0.47), indicating that datasets dominated by attractor states substantially impair the recovery of true regulatory interactions.
- **Network size (number of nodes)** showed a strong positive correlation with SHD (r = 0.63), confirming that reconstruction error increases with network complexity.
- **Trajectory length** displayed near-zero correlations with all performance metrics, suggesting that its effect is not uniform and likely depends on interactions with other factors.

These findings suggested that both data composition and network scale play important roles in reconstruction performance, motivating a multivariate analysis to disentangle their independent effects.

---

### Multivariate Regression Analysis

To quantify the independent contribution of each factor while controlling for others, we fitted ordinary least squares (OLS) regression models for structural error (SHD) and adjacency error (AHD). The primary model for SHD included attractor ratio, trajectory length, number of nodes, update mode, scoring function, and an interaction between attractor ratio and trajectory length:

```
SHD ~ attractor_ratio × trajectory_length + num_nodes + update_mode + score_function
```


In [ ]:
model = smf.ols(
    "SHD ~ attractor_ratio * trajectory_length + num_nodes + update_mode + score_function",
    data=df
).fit()

print(model.summary())

The model explained approximately 44% of the variance in SHD (R² ≈ 0.44), indicating substantial explanatory power. The following effects were statistically significant:

- **Attractor ratio** had a strong positive effect on SHD, demonstrating that a higher proportion of attractor states in the data leads to increased structural reconstruction error.
- **Update mode** (synchronous vs. asynchronous) exerted a large effect, with synchronous updates yielding significantly higher error. This result was expected, as synchronous dynamics reduce the diversity of observable transitions, effectively decreasing the informational content of the data.
- **Scoring function** had a strong effect, with MDL outperforming BDe in terms of lower structural error.
- **Trajectory length** had a smaller but statistically significant effect on SHD, indicating that longer trajectories do not necessarily improve reconstruction accuracy under fixed sample size conditions.

---

### Interaction Effects and Mechanistic Insight

To further investigate how data characteristics interact, we fitted a second regression model for adjacency error (AHD) including the interaction term:

```
AHD ~ attractor_ratio × trajectory_length + num_nodes + update_mode + score_function
```

In [ ]:
model2 = smf.ols(
    "AHD ~ attractor_ratio * trajectory_length + num_nodes + update_mode + score_function",
    data=df
).fit()
print(model2.summary())

This model revealed a significant negative interaction between attractor ratio and trajectory length, indicating that longer trajectories partially mitigate the detrimental effect of high attractor dominance. In other words, while attractor-heavy datasets are generally less informative, increasing trajectory length can partially compensate by providing more opportunities to observe transient dynamics.

---

### Summary of Determinants

Overall, reconstruction accuracy was primarily influenced by:

1. **Update mode**, which strongly affected performance due to differences in the stochasticity and informational richness of the generated trajectories. This effect is intuitive, as reduced dynamical variability limits the ability to infer causal relationships.
2. **Scoring function**, with MDL consistently outperforming BDe under the studied conditions.
3. **Attractor ratio**, which emerged as the most influential data-related factor, particularly impairing recall and increasing structural error.
4. **Network size**, which substantially increased structural error due to scaling complexity.
5. **Trajectory length**, which had a secondary, interaction-dependent effect rather than a uniform influence.

These findings demonstrate that reconstruction accuracy is jointly determined by data composition, network complexity, and inference configuration, with attractor dominance playing a central role in limiting the informativeness of time-series data.



### Visualizing the Impact of Attractor Ratio on Reconstruction Error

To complement our regression analysis and gain an intuitive understanding of how **attractor dominance interacts with scoring function and update mode**, we generated scatter plots of **SHD, AHD, BDe and MDL versus attractor ratio**, stratified by **synchronous and asynchronous updates**.

In [ ]:
category_colors = ['#C0392B', '#2980B9', '#27AE60', '#F4D03F', '#8E44AD']

plot_scatter_subplots(df1, x='attractor_ratio', y='SHD', title='SHD vs Attractor Ratio by Update Mode')
plot_scatter_subplots(df1, x='attractor_ratio', y='AHD', title='AHD vs Attractor Ratio by Update Mode')
plot_scatter_subplots(df1, x='attractor_ratio', y='BDe', title='BDE vs Attractor Ratio by Update Mode')
plot_scatter_subplots(df1, x='attractor_ratio', y='MDL', title='MDL vs Attractor Ratio by Update Mode')

##### Results:

* In the **SHD plots**, under **synchronous updates**, most points are clustered at **high attractor ratios (>0.6)**, with SHD ranging broadly from **0 to 40**, reflecting the substantial structural error associated with attractor-dominated datasets. In contrast, **asynchronous updates** show attractor ratios spanning the full range, but SHD is generally much lower for low attractor ratio trajectories, increasing gradually up to **~15** and then **~10**, confirming that asynchronous dynamics produce more informative trajectories.
* For **AHD**, synchronous updates again concentrate at **attractor ratios >0.6**, with AHD ranging between **0.1–0.8**, whereas asynchronous updates cover the full attractor range but exhibit a dense cluster near **0–0.4**, indicating overall lower adjacency errors compared to synchronous dynamics.
* For **BDe**, synchronous updates concentrate mostly at **attractor ratios >0.6, with BDe ranging between 0.0-1500**. In asynchronous updates, however, a downward trend in BDe can be observed for the atraktor_ratio range of 0.0-0.4. After that, most of the values ​​accumulate close to zero.
* For **MDE** the situation was identical to that for BDe.

Together, these scatter plots visually confirm our earlier findings: **high attractor ratios are associated with greater reconstruction errors**, and the **update mode strongly shapes the distribution of errors**, with asynchronous updates producing generally more accurate reconstructions. These visualizations provide a bridge to investigating **which factors influence attractor prevalence**, a key driver of reconstruction difficulty.

**This observation motivates the next step of our study: since attractor ratio is a key driver of reconstruction error, we now investigate which network- and data-generating factors determine the prevalence of attractor states in Boolean network trajectories.**

##### 1. Relation Between Trajectory Length and Entering Attractors

**Objective.**
To characterize how trajectory length is related to the probability of entering attractors as a function of network size and dynamics.

**Experimental design.**

- The target attractor proportion is **not controlled**; trajectories evolve naturally.
- Trajectory lengths are varied in increments proportional to network size:
    - from 10 steps to 50 by 10
- Networks are grouped by size (from 4 to 16 nodes, in steps of two).
- The number of parents per node is randomly chosen from set of $\{1,2,3\}$ to avoid conditioning results on a fixed connectivity pattern.

**Measured quantities.**

- Probability of reaching an attractor as a function of trajectory length.
- How different groups (below TODO - inner link) differ in in this probability.

**Rationale.**
Attractor entry is an emergent property of the dynamics. Controlling it directly is undesirable, as it would introduce selection bias. This experiment instead characterizes the **natural scaling behavior** of Boolean network dynamics.


In [ ]:
category_colors = ['#C0392B', '#2980B9', '#27AE60', '#F4D03F', '#8E44AD']

df_synchronous = df1[df1["update_mode"]=="synchronous"]
df_asynchronous = df1[df1["update_mode"]=="asynchronous"]

# Boxplot grouped by num_nodes synchronous
plot_boxplot(df_synchronous, x='trajectory_length', y='attractor_ratio', hue='num_nodes',
             title='Synchronous: Attractor Ratio vs Trajectory Length\nGrouped by num_nodes')
# Boxplot grouped by num_nodes synchronous
plot_boxplot(df_asynchronous, x='trajectory_length', y='attractor_ratio', hue='num_nodes',
             title='Asynchronous: Attractor Ratio vs Trajectory Length\nGrouped by num_nodes')

### Results

To further explore the factors influencing attractor prevalence, we generated boxplots of **attractor ratio versus trajectory length**, grouped by **network size (num_nodes)**, separately for **synchronous** and **asynchronous** updates.

* **Synchronous updates:** attractor ratios are generally higher across all trajectory lengths, consistent with the expected behavior of synchronous Boolean networks, where simultaneous node updates tend to drive the system more quickly into attractor states.
* **Asynchronous updates:** attractor ratios are lower overall and more widely distributed, reflecting the increased variability and longer transient dynamics inherent to asynchronous update schemes.

Across both update modes, we observe consistent trends:

1. **Trajectory length effect:** attractor ratio tends to increase with longer trajectories.
2. **Network size effect:** attractor ratio tends to decrease as the number of nodes increases.

These observations visually confirm the patterns suggested by our regression and scatter plot analyses, providing an intuitive view of how **update mode, trajectory length, and network size jointly shape attractor prevalence** in simulated Boolean networks.

### 4. Scaling Law Analysis: Normalizing Trajectory Length by Network Size

To systematically compare how trajectory length and network size jointly influence convergence, we introduced a **scaling metric**: the **scale_ratio**, defined as `trajectory_length / num_nodes`. This metric normalizes the simulation time by the network’s complexity, allowing us to identify a unified scaling behavior across different system sizes.

In [ ]:
df_summary = (
    df
    .groupby(["trajectory_length", "num_nodes"])
    .agg(
        median_ar=("attractor_ratio", "median"),
        q25_ar=("attractor_ratio", lambda x: x.quantile(0.25)),
        mean_ar=("attractor_ratio", "mean"),
        std_ar=("attractor_ratio", "std"),
        n=("attractor_ratio", "size")
    )
    .reset_index()
)

df_summary["scale_ratio"] = (
        df_summary["trajectory_length"] / df_summary["num_nodes"]
)


sns.scatterplot(
    data=df_summary,
    x="scale_ratio",
    y="median_ar",
    hue="num_nodes"
)
df_summary.head()

In [ ]:
traj_lengths = sorted(df_summary['trajectory_length'].unique())
n_cols = 3
n_rows = (len(traj_lengths) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows), sharey=True)
axes = axes.flatten()
for i, traj_len in enumerate(traj_lengths):
    ax = axes[i]
    df_sub = df_summary[df_summary['trajectory_length'] == traj_len]

    sns.scatterplot(
        data=df_sub,
        x="scale_ratio",
        y="median_ar",
        hue="num_nodes",
        palette="tab10",
        s=80,
        alpha=0.7,
        ax=ax
    )

    ax.set_title(f"Trajectory Length = {traj_len}")
    ax.set_xlabel("Scale Ratio (trajectory_length / num_nodes)")
    ax.set_ylabel("Median Attractor Ratio")
    ax.grid(True, linestyle="--", alpha=0.5)

    for spine in ax.spines.values():
        spine.set_visible(False)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.tight_layout()
fig.suptitle("Median Attractor Ratio vs Scale Ratio by Trajectory Length", fontsize=18, y=1.02)
plt.show()


##### Results

From the subplots of **median attractor ratio (`median_ar`) versus scale ratio (`trajectory_length / num_nodes`)**, several clear patterns emerge:

* **Positive relationship with scale ratio:** In general, the median attractor ratio increases as the scale ratio grows. This indicates that networks with relatively longer trajectories compared to their size tend to spend more time in attractor states.
* **Effect of trajectory length:** For larger trajectory lengths, the median attractor ratio remains high even for small scale ratios. This suggests that longer trajectories provide more opportunities for the system to reach attractor states, regardless of network size.
* **Group-level consistency:** These trends are consistent across different network sizes (`num_nodes`) and are particularly evident in groups with a sufficiently large number of observations, which ensures that the results are statistically robust.

In [ ]:
# ---- prepare dfs ----
dfs_series = dfs.copy()
dfs_series["scale"] = dfs_series["trajectory_length"] / dfs_series["num_nodes"]
dfs_series["trajectories_per_node"] = dfs_series["n_trajectories"] / dfs_series["num_nodes"]

# filtering
dfs_series = dfs_series[dfs_series["scale"] < 1.5]
dfs_series = dfs_series[dfs_series["trajectory_length"] > 1]

# ---- prepare dfa ----
dfa_series = dfa.copy()
dfa_series["scale"] = dfa_series["trajectory_length"] / dfa_series["num_nodes"]
dfa_series["trajectories_per_node"] = dfa_series["n_trajectories"] / dfa_series["num_nodes"]

# filtering
dfa_series = dfa_series[dfa_series["scale"] < 1.5]
dfa_series = dfa_series[dfa_series["trajectory_length"] > 1]

dfs['update_mode'] = 'synchronous'
dfa['update_mode'] = 'asynchronous'

dfs['k_value'] = dfs['trajectory_length'] / dfs['num_nodes']
dfa['k_value'] = dfa['trajectory_length'] / dfa['num_nodes']

dfs['k_value'] = dfs['trajectory_length'] / dfs['num_nodes']
dfa['k_value'] = dfa['trajectory_length'] / dfa['num_nodes']

# sprawdzamy jakie unikalne wartości masz
dfs['k_value'] = dfs['k_value'].round(3)
dfa['k_value'] = dfa['k_value'].round(3)

k_vals_s = sorted(dfs['k_value'].dropna().unique())
print(k_vals_s)
# [0.769, 0.778, 0.8, 0.818, 0.857]


transitions_s = [(k_vals_s[i], k_vals_s[i + 1]) for i in range(len(k_vals_s) - 1)]
print(transitions_s)
print(dfs.columns)

In [ ]:
print(dfs.columns)
df_wilcoxon_s = compute_wilcoxon_table(
    dfs,
    metrics=["AHD", "SHD"],
    transitions=transitions_s,
    sf_col='k_value'
)

print(df_wilcoxon_s)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

plot_wilcoxon_heatmap(
    df_wilcoxon_a,
    metric="AHD",
    update_mode="asynchronous",
    transitions_order=["20→40", "40→60", "60→80", "80→100"],
    ax=axes[0],
    cbar=False
)

plot_wilcoxon_heatmap(
    df_wilcoxon_a,
    metric="SID",
    update_mode="asynchronous",
    transitions_order=["20→40", "40→60", "60→80", "80→100"],
    ax=axes[1],
    cbar=True
)

fig.suptitle(
    "Paired Wilcoxon test: effect of increasing number of trajectories\n"
    "Asynchronous update",
    fontsize=18,
    y=1.05
)

plt.tight_layout()
plt.show()

### Experiment 2

In [ ]:
# Global parameters
NUM_NODES = [i for i in range(5, 17, 2)]  # od 5 do 15 (włącznie) wierzchołków, co dwie krawędzie
SCORE_FUNCTIONS = ["MDL", "BDE"]
UPDATE_MODE = ["synchronous", "asynchronous"]
N_PARENT_PER_NODE = [[1, 2, 3]]
N_REPETITIONS = 30

In [ ]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    ### paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',

    ### Tested variables
    trajectory_length=[10, 15, 20, 30, 40, 50],
    n_trajectories=[100],
    sampling_frequency=[1, 2, 3, 4, 5],

    ### Constant values per experiment
    n_repetitions=30,
    n_parents_per_node=N_PARENT_PER_NODE,

    ### Groups (bo wszędzie takie same)
    num_nodes=NUM_NODES,
    score_functions=SCORE_FUNCTIONS,
    update_mode=UPDATE_MODE,

    simulate_trajectories_to_csv_kwargs={
        # "sampling_frequency": 1,
        "target_attractor_ratio": 0.5,  # Approximate fraction of trajectory in attractor (0-1)
        "tolerance": 0.125,  # Allowed deviation from the calculated entrance step (0-1)
    }
)
# standardize sample
exp_trajectory_length.normalize_sample(100)  # WAŻNE NORMALIZUJE PRÓBĘ
exp_trajectory_length.show_experiment_df()

exp_trajectory_length.run_experiment(n_jobs=1)


##### 2. Impact of Sampling Frequency

**Objective.**
To determine how temporal subsampling affects autocorrelation, effective sample size, and reconstruction accuracy.

Dynamic Bayesian network inference assumes conditional independence of observations given parent states in the previous time slice. Excessive temporal dependence violates this assumption in practice by introducing redundant observations.

**Experimental design.**

- For fixed networks and trajectory lengths, datasets are generated using multiple sampling frequencies (1, 2, 3, 4, 5).
- For each dataset
    - ACF and ESS are computed,
    - MDL and BDe scores are extracted from BNFinder2 logs,

**Analysis.**
Accuracy is analyzed jointly as a function of:
<!-- What does it mean jointly ?? -->

* sampling frequency,
* ESS,
* scoring function (MDL or BDe).

**Rationale.**
This experiment identifies sampling regimes that balance reduced autocorrelation against loss of dynamic information due to over-subsampling.


### Experiment 3

##### 3. Amount of Trajectories Required for Stable Inference

**Objective.**
To determine how many independent trajectories are required to obtain statistically stable reconstructions.

**Experimental design.**

- Sampling frequency and trajectory length are fixed to values identified as near-optimal in previous experiments.
- The number of trajectories per dataset is gradually increased - from 10 to 100 by 10.
- For each setting, multiple (30) independent repetitions are performed to obtain convergent distribution .

**Evaluation.**

- Reconstruction accuracy is summarized using distributions (score functions).
- Stability is assessed by observing convergence of accuracy metrics as the number of trajectories increases.
- No classical parametric hypothesis test is assumed; instead, convergence trends is reported.
<!-- Nie znalazłem żadnego sensownego -->

**Rationale.**
Due to the randomness of Boolean functions and initial states, averaging over multiple networks is necessary to separate systematic effects from instance-specific variability.


In [ ]:
# create experiments:
## case: Relation between trajectory length and entering attractors
exp_trajectory_length = BooleanNetworkExperiment(
    ### paths
    data_path=DATA_PATH,
    experiment_name='trajectory_length_vs_attractors',

    ### Tested variables
    trajectory_length=list(30),
    n_trajectories=[50],
    sampling_frequency=[1],

    ### Constant values per experiment
    n_repetitions=N_REPETITIONS,
    n_parents_per_node=N_PARENT_PER_NODE,

    ### Groups (bo wszędzie takie same)
    num_nodes=NUM_NODES,
    score_functions=SCORE_FUNCTIONS,
    update_mode=UPDATE_MODE,

    simulate_trajectories_to_csv_kwargs={
        # "sampling_frequency": 1,
        "target_attractor_ratio": 0.5,  # Approximate fraction of trajectory in attractor (0-1)
        "tolerance": 0.5,  # Allowed deviation from the calculated entrance step (0-1)
    }
)
exp_trajectory_length.show_experiment_df()

### Experiment 4

# Part 2

## Methodology

## Analysis